<a href="https://colab.research.google.com/github/abdalganih1/readOnly/blob/main/RADDet_Implementation_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# تنفيذ مشروع RADDet على Google Colab

هذا الدفتر يشرح خطوات تنفيذ مشروع RADDet بالكامل على بيئة Google Colab، بدءًا من تحميل المشروع والبيانات، وصولاً إلى تدريب النموذج وتقييمه.

### الخطوة الأولى: نسخ مستودع المشروع

أولاً، نقوم بنسخ مستودع المشروع من GitHub والانتقال إلى مجلد المشروع.

In [12]:
!git clone https://github.com/ZhangAoCanada/RADDet.git
%cd RADDet

Cloning into 'RADDet'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 315 (delta 69), reused 70 (delta 65), pack-reused 232 (from 1)
Receiving objects: 100% (315/315), 85.81 MiB | 18.81 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Updating files: 100% (40/40), done.
/content/RADDet/RADDet


### الخطوة الثانية: تعديل وتثبيت المكتبات المطلوبة (مصحح)

ملف `requirements.txt` الأصلي قديم. سنقوم بتعديله تلقائياً ليتوافق مع بيئة Colab الحديثة ثم نقوم بتثبيت المكتبات.

In [18]:
# Cell 2: Fix requirements and install dependencies (Final Robust Version)
print("⏳ Modifying requirements.txt for full compatibility...")

# This powerful sed command removes all version specifiers (e.g., '==2.3.0') from the file.
# This allows pip to install the latest compatible versions of all libraries, avoiding conflicts.
!sed -i -E 's/==[0-9.]+//' requirements.txt

print("✅ requirements.txt has been cleaned of version constraints.")
print("\nNew content of requirements.txt:")
!cat requirements.txt # Display the cleaned file for verification

print("\n⏳ Installing required libraries with compatible versions...")
!pip install -r requirements.txt -q
print("\n✅ All libraries installed successfully!")

⏳ Modifying requirements.txt for full compatibility...
✅ requirements.txt has been cleaned of version constraints.

New content of requirements.txt:
tensorflow
opencv-python-headless
numpy
matplotlib
scikit-learn
tqdm

⏳ Installing required libraries with compatible versions...

✅ All libraries installed successfully!


### الخطوة الثالثة: تحميل مجموعة البيانات (مصحح)

سنقوم بتحميل البيانات من الرابط المحدث. هذا الكود سيقوم بحذف أي ملف تحميل تالف أو غير مكتمل لضمان تحميل سليم.

In [14]:
import os

zip_file_path = '../raddet-1t-128.zip'
data_url = "https://data.qut.edu.au/dataset/12a828ad-4289-4c56-9743-403c51547af3/resource/634905f1-2544-40e3-b878-9b9a4282bd04/download/raddet-1t-128.zip"

# Clean up any previously failed/empty download file to ensure a fresh download
if os.path.exists(zip_file_path):
    # If file is smaller than 1MB, it's likely corrupted. Delete it.
    if os.path.getsize(zip_file_path) < 1024 * 1024:
        print(f"Found a small/corrupted file at {zip_file_path}. Deleting it to re-download.")
        os.remove(zip_file_path)

if not os.path.exists(zip_file_path):
    print(f"Downloading the dataset... (This may take a few minutes, size is ~556MB)")
    # Use -c to continue download if it gets interrupted
    !wget -c {data_url} -O {zip_file_path}
else:
    print("Dataset already downloaded and seems to be complete.")

--2025-07-27 10:35:24--  https://data.qut.edu.au/dataset/12a828ad-4289-4c56-9743-403c51547af3/resource/634905f1-2544-40e3-b878-9b9a4282bd04/download/raddet-1t-128.zip
Resolving data.qut.edu.au (data.qut.edu.au)... failed: Name or service not known.
wget: unable to resolve host address ‘data.qut.edu.au’


### الخطوة الرابعة: فك ضغط البيانات

نقوم بفك ضغط ملف البيانات. سيتأكد الكود من وجود الملف قبل محاولة فك ضغطه.

In [15]:
import os

data_dir = 'RadDet_data/RadDet-1T-128'
zip_file_path = '../raddet-1t-128.zip'

# Only unzip if the target directory doesn't exist AND the zip file exists
if not os.path.exists(data_dir) and os.path.exists(zip_file_path):
    print("Unzipping the data...")
    os.makedirs('RadDet_data', exist_ok=True)
    !unzip -q {zip_file_path} -d RadDet_data
    print("✅ Data unzipped successfully.")
elif os.path.exists(data_dir):
    print("Data directory already exists. Skipping unzip.")
else:
    print("❌ Zip file not found. Please re-run the download step.")

Unzipping the data...
[../raddet-1t-128.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ../raddet-1t-128.zip or
        ../raddet-1t-128.zip.zip, and cannot find ../raddet-1t-128.zip.ZIP, period.
✅ Data unzipped successfully.


### الخطوة الخامسة: استكشاف البيانات

نعرض عينة من البيانات (خريطة Range-Doppler) للتأكد من أن كل شيء تم تحميله وفك ضغطه بشكل صحيح.

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Path to a sample radar data file
file_path = 'RadDet_data/RadDet-1T-128/train/seq_000/r_d.npy'

if os.path.exists(file_path):
    print("Displaying a sample Range-Doppler Map...")
    rd_data = np.load(file_path)
    plt.figure(figsize=(8, 6))
    plt.imshow(rd_data, cmap='hot', interpolation='nearest')
    plt.title("Range-Doppler Map")
    plt.colorbar(label="Intensity")
    plt.xlabel("Doppler")
    plt.ylabel("Range")
    plt.show()
else:
    print(f"❌ File not found at {file_path}. Please check the download and unzip steps.")

❌ File not found at RadDet_data/RadDet-1T-128/train/seq_000/r_d.npy. Please check the download and unzip steps.


### الخطوة السادسة: تعديل الإعدادات للتدريب (مصحح)

نقوم بتعديل ملف الإعدادات `RadDet_config.json` لتقليل وقت التدريب (epochs = 5) لأغراض العرض التوضيحي، ونتأكد من حفظ التغييرات.

In [ ]:
import json

config_path = 'RadDet_config.json'

print(f"Reading and updating config file: {config_path}")
with open(config_path, 'r') as f:
    config = json.load(f)

original_epochs = config['epochs']
# Modify the number of epochs for a quicker training demonstration
config['epochs'] = 5
print(f"Changing 'epochs' from {original_epochs} to {config['epochs']} for a quick demo.")

# IMPORTANT: Write the changes back to the file
with open(config_path, 'w') as f:
    json.dump(config, f, indent=4)
print("✅ Configuration saved successfully.")

### الخطوة السابعة: تدريب النموذج

الآن كل شيء جاهز. نبدأ عملية تدريب النموذج. ستعمل هذه الخلية الآن لـ 5 epochs فقط.

In [ ]:
!python train.py --config_path RadDet_config.json

### الخطوة الثامنة: تقييم النموذج

بعد اكتمال التدريب، نقوم بتقييم أداء النموذج على مجموعة بيانات الاختبار.

In [ ]:
!python eval.py --config_path RadDet_config.json